# Data Science job posting on Glassdoor 
##     Data cleaning and Transformation

### Questions to answer
##### * Can you make the salary column into integers?
##### * What information can you extract out of job descriptions?
##### * How can you remove the numbers from the company name?
##### * How can you create some new features? (e.g. state column from the location column)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\K A L K I D A N\OneDrive\Desktop\projects\data cleaning\glassdoor_jobs.CSV")

In [3]:
df.info

<bound method DataFrame.info of                                              Job Title  \
0                                       Data Scientist   
1                                       Data Scientist   
2                                       Data Scientist   
3                                       Data Scientist   
4                                       Data Scientist   
..                                                 ...   
995                                     Data Scientist   
996  Analytic Consultant 4 - Branch Banking Reporti...   
997                                     Data Scientist   
998                                     Data Scientist   
999                              Clinical Data Analyst   

                  Salary Estimate  \
0      $77K-$92K (Glassdoor Est.)   
1      $77K-$92K (Glassdoor Est.)   
2      $77K-$92K (Glassdoor Est.)   
3      $77K-$92K (Glassdoor Est.)   
4      $77K-$92K (Glassdoor Est.)   
..                            ...   
995  $105K-$

In [4]:
# Remove the '$' sign, 'K' for thousands, and extract the lower and upper ranges
df['Salary Estimate'] = df['Salary Estimate'].str.replace(r'[^0-9\-]', '', regex=True)

# Split the salary range and calculate the average salary
df['Salary Estimate'] = df['Salary Estimate'].str.split('-').apply(lambda x: (int(x[0]) + int(x[1])) / 2 if len(x) == 2 else int(x[0]))

# Convert to integer
df['Salary Estimate'] = df['Salary Estimate'].astype(int)


In [5]:
df['Salary Estimate']

0       84
1       84
2       84
3       84
4       84
      ... 
995    121
996    121
997    121
998    121
999    121
Name: Salary Estimate, Length: 1000, dtype: int32

In [6]:
import re

# Extract skills from job descriptions (example with a few keywords)
df['skills'] = df['Job Description'].apply(lambda x: ', '.join(re.findall(r'\b(Python|SQL|Machine Learning|Data Analysis|R)\b', x, flags=re.IGNORECASE)))

# Example: Extracting years of experience from job descriptions
df['years_of_experience'] = df['Job Description'].apply(lambda x: re.findall(r'\d+ years? of experience', x))

# Example: Extracting if the education level is mentioned (Bachelor or Master)
df['education_required'] = df['Job Description'].apply(lambda x: 'Bachelor' if 'bachelor' in x.lower() else ('Master' if 'master' in x.lower() else 'None'))


In [7]:
# Remove numbers from the company name
df['Company Name'] = df['Company Name'].replace({r'\d+': ''}, regex=True).str.strip()


In [8]:
df['Company Name']

0      Affinity Solutions\n.
1                 ManTech\n.
2                  Takeda\n.
3         A Place for Mom\n.
4                 GovTech\n.
               ...          
995          Shelf Engine\n.
996           Wells Fargo\n.
997            Fannie Mae\n.
998    XSELL Technologies\n.
999                NYSTEC\n.
Name: Company Name, Length: 1000, dtype: object

In [11]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,skills,years_of_experience,education_required
0,Data Scientist,84,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n.,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee","machine learning, machine learning, Python, R,...",[],None
1,Data Scientist,84,"Secure our Nation, Ignite your Future\n\nSumma...",4.1,ManTech\n.,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,₹100 to ₹500 billion (INR),-1,"data analysis, data analysis, machine learning...",[],None
2,Data Scientist,84,"By clicking the Apply button, I understand tha...",3.7,Takeda\n.,"Cambridge, MA","OSAKA, Japan",10000+ employees,1781,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,₹500+ billion (INR),"Novartis, Baxter, Pfizer","Machine Learning, machine learning, Data Analy...",[],Master
3,Data Scientist,84,Join Our Growing Team\nA career with A Place f...,3.0,A Place for Mom\n.,"Overland Park, KS","New York, NY",501 to 1000 employees,2000,Company - Private,Healthcare Services & Hospitals,Healthcare,Unknown / Non-Applicable,"Enlivant, Sunrise Senior Living, Brookdale Sen...","machine learning, machine learning, data analy...",[],Bachelor
4,Data Scientist,84,We are looking for Data Scientists who are int...,3.7,GovTech\n.,"San Francisco, CA","Singapore, Singapore",1001 to 5000 employees,2016,Government,Government Agencies,Government,Unknown / Non-Applicable,-1,"machine learning, R, Python",[],Bachelor


In [12]:
# First, let's check if the Location column has the expected format
df['Location'].head()

# Split the 'Location' column into 'City' and 'State' carefully
df[['City', 'State']] = df['Location'].str.split(',', expand=True, n=1)

# Clean up extra spaces and handle missing values
df['City'] = df['City'].str.strip()
df['State'] = df['State'].str.strip()

# Check for any rows with missing values (in case some locations do not follow the 'City, State' format)
df[['City', 'State']].isnull().sum()

# For rows where the State is missing, assign the Country (if available) to the State
df['State'].fillna(df['Location'].apply(lambda x: x.split(',')[-1] if ',' in x else ''), inplace=True)

# Display the updated columns
print(df[['Location', 'City', 'State']].head())


            Location           City State
0       New York, NY       New York    NY
1      Chantilly, VA      Chantilly    VA
2      Cambridge, MA      Cambridge    MA
3  Overland Park, KS  Overland Park    KS
4  San Francisco, CA  San Francisco    CA


C:\Users\K A L K I D A N\AppData\Local\Temp\ipykernel_10508\3972090298.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['State'].fillna(df['Location'].apply(lambda x: x.split(',')[-1] if ',' in x else ''), inplace=True)


In [13]:
df.info

<bound method DataFrame.info of                                              Job Title  Salary Estimate  \
0                                       Data Scientist               84   
1                                       Data Scientist               84   
2                                       Data Scientist               84   
3                                       Data Scientist               84   
4                                       Data Scientist               84   
..                                                 ...              ...   
995                                     Data Scientist              121   
996  Analytic Consultant 4 - Branch Banking Reporti...              121   
997                                     Data Scientist              121   
998                                     Data Scientist              121   
999                              Clinical Data Analyst              121   

                                       Job Description  Rating  \
0

In [14]:
# To clean the company names by removing unwanted characters like \n, we can use the str.replace() method.
df['Company Name'] = df['Company Name'].str.replace(r'\n|\.','', regex=True)


In [19]:
df['Company Name']

0      Affinity Solutions
1                 ManTech
2                  Takeda
3         A Place for Mom
4                 GovTech
              ...        
995          Shelf Engine
996           Wells Fargo
997            Fannie Mae
998    XSELL Technologies
999                NYSTEC
Name: Company Name, Length: 1000, dtype: object

In [15]:
# create a new column for Skills by identifying the common skills mentioned:
df['Skills'] = df['Job Description'].str.extract(r'(machine learning|Python|R|SQL|data analysis)', expand=False)


In [18]:
df['Skills'] 

0      machine learning
1         data analysis
2      machine learning
3                     R
4      machine learning
             ...       
995                 SQL
996                   R
997                   R
998                   R
999                   R
Name: Skills, Length: 1000, dtype: object

In [16]:
#  Create a new column that groups companies into Small, Medium, or Large based on the number of employees.
df['Company Size Category'] = df['Size'].apply(lambda x: 'Small' if '1 to 50' in x else ('Medium' if '51 to 200' in x or '201 to 1000' in x else 'Large'))


In [20]:
df['Company Size Category']

0      Medium
1       Large
2       Large
3       Large
4       Small
        ...  
995     Small
996     Large
997     Large
998    Medium
999    Medium
Name: Company Size Category, Length: 1000, dtype: object

In [17]:
df.info

<bound method DataFrame.info of                                              Job Title  Salary Estimate  \
0                                       Data Scientist               84   
1                                       Data Scientist               84   
2                                       Data Scientist               84   
3                                       Data Scientist               84   
4                                       Data Scientist               84   
..                                                 ...              ...   
995                                     Data Scientist              121   
996  Analytic Consultant 4 - Branch Banking Reporti...              121   
997                                     Data Scientist              121   
998                                     Data Scientist              121   
999                              Clinical Data Analyst              121   

                                       Job Description  Rating  \
0